指定したフォルダ内のwav音声全てからPNデータを導出してDBに記録する（発話内容は事前に指定）

In [ ]:
import tkinter as tk
from tkinter import filedialog
# ルートウィンドウ作成
root = tk.Tk()
# ルートウィンドウの非表示
root.withdraw()
idir =  './' #カレントディレクトリ
# idir =  '/Users/kentatakemae/Desktop/Workplace/Project_PopNoise_python'
dir_path = tk.filedialog.askdirectory(initialdir = idir)

print(dir_path)

In [ ]:
#同じ発話内容のWav発話音声を入力し、それぞれに対応するtextファイルを自動生成する
#すべてのwavファイルをコピーしてダウンサンプリングしたものと、作成したtextファイルを全てjulius-segmentation-kitのwavフォルダに入れる

import os, glob
import librosa
import soundfile as sf
import pandas as pd


phrase = 'そんしつにめげないこうみん'

# #後で消す！！！---------------------------------------------------------------------
# dir_path = 'input_data' 
# #---------------------------------------------------------------------------------


# # ワイルドカードで条件を満たすパスの文字列を指定
# pathlist = glob.glob(dir_path + '/*.wav')

# 全てのwavファイルのパスをpathlistに格納 / **の部分があらゆる中間ディレクトリに対応
pathlist = glob.glob(dir_path  + '/**/*.wav', recursive=True)




#各wav音声ファイルに対応するテキストファイルを作成する / wavファイルをJulius用にダウンサンプリング（44.1kHz　to 16kHz）する
for path_name in pathlist:
    print(path_name)
    file_name = os.path.splitext(os.path.basename(path_name))[0]
    print(file_name)

    #テキストファイルをまとめて入れるためのフォルダを作成する
    new_txt_path = 'wav/{}.txt'.format(file_name)
    f = open(new_txt_path, 'w', encoding = 'utf8')
    f.write('{}'.format(phrase)) #日本語フレーズをテキストに記入
    f.close()

    # サンプリングレート44.1kHzのwavを16kHzに変換する
    y, sr = librosa.core.load(path_name, sr=16000, mono=True) # 16kHz、モノラルで読み込み
    new_wav_path = 'wav/{}.wav'.format(file_name)
    sf.write(new_wav_path, y, sr, subtype="PCM_16") #16bitで書き込み


In [ ]:
#Julius-segmentationを実行
import subprocess

# o = subprocess.run(['perl','test.pl'],stdout = subprocess.PIPE, stderr = subprocess.PIPE, shell = True)
# o.stdout
subprocess.run(['perl','segment_julius.pl'], stdout = subprocess.PIPE, stderr = subprocess.PIPE,shell = True)

In [ ]:
#実行結果の「.lab」ファイルと元の音源(44.1kHz)からPNデータを導出し、DBに記録する。

#自作のPND関数を呼び出す
import my_func

# # ファイル名のみを取得
# namelist = [os.path.basename(p) for p in glob.glob(dir_path + '/**', recursive=True) if os.path.isfile(p)]

pho_PNene = pd.DataFrame()
for path_name in pathlist:
    # file_name = os.path.splitext(os.path.basename(path_name))[0]
    # print(file_name)
    result = my_func.pnd(path_name)
    # print(pho_PNene)
    # print(result['PN-energy'])
    pho_PNene = pd.concat([pho_PNene,result['PN-energy']], axis=1)


import shutil
shutil.rmtree('output') #
os.mkdir('output') #新しいフォルダの作成
pho_PNene['PN-energy'].T.to_excel('output/output.xlsx',index=False, header=False)

#DBのパスを指定


In [ ]:
#処理中に作成したファイルを全て削除する
import shutil
shutil.rmtree('wav')

#新しいフォルダの作成
os.mkdir('wav')